# Search embedding

## Load dataset

In [1]:
# Here we store constant varables with path to our datasets
# SEARCH_DATA_INDEX_PATH = "../../datasets" #"../../datasets/metadata_for_search"
# MODEL_PATH = "../../models/"

# SEARCH_DATA_PATH = "../../datasets"
# Here we store constant varables with path to our datasets
SEARCH_DATA_INDEX_PATH = "../../datasets/processed_datasets"
MODEL_PATH = "../../models/"

SEARCH_DATA_PATH = "../../datasets"

#### load search data from kaggle

In [2]:
import pandas as pd
# here we load metadata (related to source of dataset)
data_vote = pd.read_json(SEARCH_DATA_PATH+"/search_dataset_vote.json")
data_active = pd.read_json(SEARCH_DATA_PATH+"/search_dataset_active.json")
data_hot = pd.read_json(SEARCH_DATA_PATH+"/search_dataset_hot.json")

# here we load datasets (texts)
search_data_index_hot = pd.read_csv(SEARCH_DATA_INDEX_PATH+"/metadata-for-search-info-hottest-rating.csv")
search_data_index_vote = pd.read_csv(SEARCH_DATA_INDEX_PATH+"/metadata-for-search-info.csv")
search_data_index_active = pd.read_csv(SEARCH_DATA_INDEX_PATH+"/metadata-for-search-info-active-rating.csv")

In [3]:
data_hot

,path,id,title,subtitle,description,keywords,titleNullable,subtitleNullable,descriptionNullable,text
0,../datasets/raw_datasets/metadata-for-search/o...,0,Top 10000 Popular Movies Dataset,Top 10000 Popular movies based on TMDB ratings,### Context\n\nRecommendation systems are used...,"[arts and entertainment, movies and tv shows, ...",Top 10000 Popular Movies Dataset,Top 10000 Popular movies based on TMDB ratings,### Context\n\nRecommendation systems are used...,Top 10000 Popular Movies Dataset Top 10000 Po...
1,../datasets/raw_datasets/metadata-for-search/r...,1,IMDB Top 1000 Movies Dataset,"""Release Year, Duration, Ratings, Metascores, ...",This dataset provides comprehensive informatio...,[movies and tv shows],IMDB Top 1000 Movies Dataset,"""Release Year, Duration, Ratings, Metascores, ...",This dataset provides comprehensive informatio...,"IMDB Top 1000 Movies Dataset ""Release Year, D..."
2,../datasets/raw_datasets/metadata-for-search/i...,2,Top 1000 IMDb Movies Dataset,Discover the Greatest Movies of All Time - IMD...,"Welcome to the ""Top 1000 IMDb Movies Dataset"" ...",[movies and tv shows],Top 1000 IMDb Movies Dataset,Discover the Greatest Movies of All Time - IMD...,"Welcome to the ""Top 1000 IMDb Movies Dataset"" ...",Top 1000 IMDb Movies Dataset Discover the Gre...
3,../datasets/raw_datasets/metadata-for-search/u...,3,Top Rated 10000 Movies Dataset (IMDB),In this dataset the top movies data is given b...,\nIn this dataset all time top movies data is...,[movies and tv shows],Top Rated 10000 Movies Dataset (IMDB),In this dataset the top movies data is given b...,\nIn this dataset all time top movies data is...,Top Rated 10000 Movies Dataset (IMDB) In this...
4,../datasets/raw_datasets/metadata-for-search/s...,4,Top 1000 Highest Grossing Movies,Top 1000 Highest Grossing Holywood Movies,Context\nThis dataset contains information abo...,"[movies and tv shows, beginner, intermediate, ...",Top 1000 Highest Grossing Movies,Top 1000 Highest Grossing Holywood Movies,Context\nThis dataset contains information abo...,Top 1000 Highest Grossing Movies Top 1000 Hig...
...,...,...,...,...,...,...,...,...,...,...
684,../datasets/raw_datasets/metadata-for-search/n...,685,Scene Classification,Contains ~25K images from a wide range of natu...,This dataset contains about ~25k images from a...,"[movies and tv shows, computer vision, classif...",Scene Classification,Contains ~25K images from a wide range of natu...,This dataset contains about ~25k images from a...,Scene Classification Contains ~25K images fro...
685,../datasets/raw_datasets/metadata-for-search/m...,686,Plants Classification,30 Types of Plants Image Classification,The dataset contains 30 types of plants images...,"[biology, classification, image]",Plants Classification,30 Types of Plants Image Classification,The dataset contains 30 types of plants images...,Plants Classification 30 Types of Plants Imag...
686,../datasets/raw_datasets/metadata-for-search/m...,687,Flower Classification,14 Types of Flower Image Classification,The dataset contains 14 types of flower images...,"[classification, image, chinese]",Flower Classification,14 Types of Flower Image Classification,The dataset contains 14 types of flower images...,Flower Classification 14 Types of Flower Imag...
687,../datasets/raw_datasets/metadata-for-search/g...,688,BIRDS 525 SPECIES- IMAGE CLASSIFICATION,"525 species, 84635 train, 2625 test, 2625 vali...",This version of the dataset adds 10 new specie...,"[biology, computer vision, classification, cnn...",BIRDS 525 SPECIES- IMAGE CLASSIFICATION,"525 species, 84635 train, 2625 test, 2625 vali...",This version of the dataset adds 10 new specie...,BIRDS 525 SPECIES- IMAGE CLASSIFICATION 525 ...


In [4]:
data_hot['id'], data_hot['text']

(0        0
 1        1
 2        2
 3        3
 4        4
       ... 
 684    685
 685    686
 686    687
 687    688
 688    689
 Name: id, Length: 689, dtype: int64,
 0       Top 10000 Popular Movies Dataset Top 10000 Po...
 1       IMDB Top 1000 Movies Dataset "Release Year, D...
 2       Top 1000 IMDb Movies Dataset Discover the Gre...
 3       Top Rated 10000 Movies Dataset (IMDB) In this...
 4       Top 1000 Highest Grossing Movies Top 1000 Hig...
                              ...                        
 684     Scene Classification Contains ~25K images fro...
 685     Plants Classification 30 Types of Plants Imag...
 686     Flower Classification 14 Types of Flower Imag...
 687     BIRDS 525  SPECIES- IMAGE CLASSIFICATION 525 ...
 688     Brain Tumor Classification (MRI) Classify MRI...
 Name: text, Length: 689, dtype: object)

In [5]:
queries = search_data_index_vote['query name'].unique()

#### load dataset Ubuntu questions

In [6]:
from datasets import load_dataset

ask_dataset = load_dataset("mteb/askubuntudupquestions-reranking",)

In [7]:
ask_dataset

DatasetDict({
    test: Dataset({
        features: ['query', 'positive', 'negative'],
        num_rows: 375
    })
})

In [8]:
def process_dataset(sample):
    sample['responce'] = sample['positive']+sample['negative']
    sample['target'] = list(range(len(sample['responce'])))
    return sample
ask_dataset1 = ask_dataset.map(process_dataset)

## Models

### Inference batching utils

In [9]:
# this function return batches of our datasets
def iterate_batch(dataset,batch_size=128):
    for i in range(0,len(dataset),batch_size):
        yield dataset[i:i+batch_size]

In [10]:
iterate_batch([1,2,3,4,5,6,7,8,9,10],3)

<generator object iterate_batch at 0x7f52ac12a570>

### qween

In [11]:
import torch
from typing import Optional, cast

import numpy as np
import numpy.typing as npt
from chromadb.api.types import EmbeddingFunction, Documents, Embeddings

from torch import Tensor
from transformers import AutoTokenizer, AutoModel


class TransformerEmbeddingFunction_Qween(EmbeddingFunction[Documents]):    
    """
    The TransformerEmbeddingFunction_Qween class is an embedding function
    that uses a pre-trained transformer model to generate embeddings for a
    given set of documents. It is designed to work with the ChromaDB
    library for building vector search engines.
    """
    def __init__(self):
        """
        Load a pre-trained tokenizer and model from the 'Alibaba-NLP/gte-Qwen2-7B-instruct' model
        checkpoint
        It also sets the max_length parameter, which is the maximum length of input sequences
        that the model can handle.
        """
        self.tokenizer = AutoTokenizer.from_pretrained('Alibaba-NLP/gte-Qwen2-7B-instruct', trust_remote_code=True)
        self.model = AutoModel.from_pretrained('Alibaba-NLP/gte-Qwen2-7B-instruct', trust_remote_code=True)
        self.model = self.model.to(device)
        self.model.eval()
        
        self.max_length = 8192
        return
    def __call__(self, input: Documents) -> Embeddings:
        """
        The __call__ method is the main entry point for generating embeddings. It takes a list of documents as input and returns a list of embeddings.
        It first defines a helper function called last_token_pool that extracts the embedding of the last token from the model's output. This is done by checking if the input sequence has left padding and selecting the appropriate token index.
        The input documents are tokenized using the tokenizer and passed through the model to obtain the last hidden states.
        The last_token_pool function is applied to the last hidden states to extract the final embeddings.
        The embeddings are converted to a list and returned.
        """
        
        def last_token_pool(last_hidden_states: Tensor,attention_mask: Tensor) -> Tensor:
            left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
            if left_padding:
                return last_hidden_states[:, -1]
            else:
                sequence_lengths = attention_mask.sum(dim=1) - 1
                batch_size = last_hidden_states.shape[0]
                return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]
        embeddings = 0
        with torch.no_grad():
            for batch in iterate_batch(input,3):
                embedding=0
            
                batch_dict = self.tokenizer(batch, max_length=self.max_length, padding=True, truncation=True, return_tensors='pt').to(device)
                outputs = self.model(**batch_dict)
                embedding = last_token_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
                
                if type(embeddings) is int:
                        embeddings = embedding
                else:
                    embeddings = torch.cat((embeddings,embedding),dim=0)
                torch.cuda.empty_cache()

        

        
        return embeddings.tolist()
    
    def preprocess_query(self,text_list):
        """
        The preprocess_query method is used to preprocess the input queries before generating embeddings.
        It defines a task description and a helper function called get_detailed_instruct that appends the task description to each query.
        The method applies the get_detailed_instruct function to each query in the input list and returns the preprocessed queries.
        Overall, this code provides a convenient way to use a pre-trained transformer model for generating document embeddings in the context of a vector search engine built with ChromaDB.        
        """
        
        task = 'Given a web search query, retrieve relevant passages that answer the query'
        def get_detailed_instruct(task_description: str, query: str) -> str:
            return f'Instruct: {task_description}\nQuery: {query}'

        text_list = [get_detailed_instruct(task, text) for text in text_list]
        return text_list

### Load vector database for kaggle

In [12]:
device = 'cuda'
model_embed = TransformerEmbeddingFunction_Qween()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [13]:
all_results_hot = []
all_results_vote = []
all_results_active = []

In [14]:
metrics = {}

In [15]:
model_name = "qween"

In [16]:
load_time = 0
search_time = 0

In [17]:
# client.delete_collection("seach_db_vote")
# client.delete_collection("seach_db_active")
# client.delete_collection("seach_db_hot")

In [18]:
import chromadb
client = chromadb.Client()

In [19]:
import chromadb
import time

metrics[model_name] = {"active":{},"vote":{},"hot":{}}

client = chromadb.Client()

collection_vote = client.create_collection("seach_db_vote",get_or_create=False,embedding_function=model_embed)
collection_active = client.create_collection("seach_db_active",get_or_create=False,embedding_function=model_embed)
collection_hot = client.create_collection("seach_db_hot",get_or_create=False,embedding_function=model_embed)
# collection = client.create_collection("test_seach_fb",embedding_function,get_or_create=True)

# load
load_time = time.time()
collection_vote.add(
    documents=list(data_vote['text']),
    ids=[str(i) for i in data_vote['id']]
)
load_time = time.time() - load_time
metrics[model_name]['vote']['load_time'] = load_time

load_time = time.time()
collection_active.add(
    documents=list(data_active['text']),
    ids=[str(i) for i in data_active['id']]
)
load_time = time.time() - load_time
metrics[model_name]['active']['load_time'] = load_time

load_time = time.time()
collection_hot.add(
    documents=list(data_hot['text']),
    ids=[str(i) for i in data_hot['id']]
)
load_time = time.time() - load_time
metrics[model_name]['hot']['load_time'] = load_time

In [20]:
search_time = time.time()
queries = list(search_data_index_vote['query name'].unique())
queries = model_embed.preprocess_query(queries)
results_vote = collection_vote.query(
    query_texts=queries, # Chroma will embed this for you
    n_results=40 # how many results to return
)
search_time = time.time() - search_time
metrics[model_name]['vote']['search_time'] = search_time

search_time = time.time()
queries = list(search_data_index_hot['query name'].unique())
queries = model_embed.preprocess_query(queries)
results_hot = collection_hot.query(
    query_texts=queries, # Chroma will embed this for you
    n_results=40 # how many results to return
)
search_time = time.time() - search_time
metrics[model_name]['hot']['search_time'] = search_time

search_time = time.time()
queries = list(search_data_index_active['query name'].unique())
queries = model_embed.preprocess_query(queries)
results_active = collection_active.query(
    query_texts=queries, # Chroma will embed this for you
    n_results=40 # how many results to return
)
search_time = time.time() - search_time
metrics[model_name]['active']['search_time'] = search_time

client.delete_collection("seach_db_vote")
client.delete_collection("seach_db_active")
client.delete_collection("seach_db_hot")

In [21]:
ask_dataset1

DatasetDict({
    test: Dataset({
        features: ['query', 'positive', 'negative', 'responce', 'target'],
        num_rows: 375
    })
})

### load vector base for ask ubuntu

In [22]:
import chromadb
import time

metrics2[model_name] = {"ubuntu":{}}

client = chromadb.Client()

collection_ubuntu = client.create_collection("seach_db_ubuntu",get_or_create=False,embedding_function=model_embed)
# collection = client.create_collection("test_seach_fb",embedding_function,get_or_create=True)

# load
load_time = time.time()
collection_vote.add(
    documents=list(ask_dataset1['responce'])
)
load_time = time.time() - load_time
metrics[model_name]['vote']['load_time'] = load_time

NameError: name 'metrics2' is not defined

In [ ]:
search_time = time.time()
queries = list(search_data_index_vote['query name'].unique())
queries = model_embed.preprocess_query(queries)
results_vote = collection_vote.query(
    query_texts=queries, # Chroma will embed this for you
    n_results=40 # how many results to return
)
search_time = time.time() - search_time
metrics[model_name]['vote']['search_time'] = search_time

client.delete_collection("seach_db_hot")

## Evaluation


In [23]:
import sys
sys.path.insert(0, '../../scripts/')

In [ ]:
from evaluation import evaluate_algs

In [27]:
metrics[model_name]['vote']['metrics'] = evaluate_algs(results_vote,search_data_index_vote,column_name='rating')
metrics[model_name]['active']['metrics'] = evaluate_algs(results_active,search_data_index_active,column_name='rating_active')
metrics[model_name]['hot']['metrics'] = evaluate_algs(results_hot,search_data_index_hot,column_name='rating_hottest')

/home/danil/Text_search_with_LLM/sample/lib/python3.10/site-packages/ranx/metrics/recall.py:29: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  scores[i] = _recall(qrels[i], run[i], k, rel_lvl)


In [28]:
metrics[model_name]['vote']['metrics']

{'recall@10': 0.2288268800739379,
 'precision@10': 0.5782608695652174,
 'hits@10': 5.782608695652174,
 'map@10': 0.18642717258326347,
 'mrr@10': 0.7815217391304349,
 'ndcg@10': 0.390860835402863,
 'recall@40': 0.5568091062932357,
 'precision@40': 0.46141304347826095,
 'hits@40': 18.456521739130434,
 'map@40': 0.40018413730541147,
 'mrr@40': 0.7826659038901601,
 'ndcg@40': 0.48348643757160337}

In [29]:
metrics[model_name]['active']['metrics']

{'recall@10': 0.537359022556391,
 'precision@10': 0.531578947368421,
 'hits@10': 5.315789473684211,
 'map@10': 0.42852861319966573,
 'mrr@10': 0.7907894736842105,
 'ndcg@10': 0.5269621411880194,
 'recall@40': 0.8498746867167922,
 'precision@40': 0.2513157894736842,
 'hits@40': 10.052631578947368,
 'map@40': 0.6052456815282098,
 'mrr@40': 0.7953748006379586,
 'ndcg@40': 0.6456429264912914}

In [30]:
metrics[model_name]['hot']['metrics']

{'recall@10': 0.4683898803216984,
 'precision@10': 0.6249999999999999,
 'hits@10': 6.25,
 'map@10': 0.3976503272160739,
 'mrr@10': 0.8358585858585859,
 'ndcg@10': 0.44386353572764725,
 'recall@40': 0.8556416437098256,
 'precision@40': 0.32784090909090907,
 'hits@40': 13.113636363636363,
 'map@40': 0.6334733570628028,
 'mrr@40': 0.8369408369408369,
 'ndcg@40': 0.6110369817475614}